In [13]:
import numpy as np
import pandas as pd
import librosa
import matplotlib.pyplot as plt
from keras.utils import to_categorical
import os
import sys
from tqdm import tqdm
import wavio
import shutil
shutil.rmtree('npy-files')
os.mkdir("npy-files")
import csv

In [17]:
def chop_array(arr, window_size, hop_size):
    """chop_array([1,2,3], 2, 1) -> [[1,2], [2,3]]"""
    return [arr[i - window_size:i] for i in range(window_size, len(arr) + 1, hop_size)]

def safe_log(x):
    """Prevents error on log(0) or log(-1)"""
#     return np.log2(np.clip(x, (np.finfo(np.float16).eps), None))
    return np.log2(x)

def filterbanks(sample_rate, num_filt, fft_len):
    """Makes a set of triangle filters focused on {num_filter} mel-spaced frequencies"""
    def hertz_to_mels(f):
        return 1127. * np.log(1. + f / 700.)

    def mel_to_hertz(mel):
        return 700. * (np.exp(mel / 1127.) - 1.)

    def correct_grid(x):
        """Push forward duplicate points to prevent useless filters"""
        offset = 0
        for prev, i in zip([x[0] - 1] + x, x):
            offset = max(0, offset + prev + 1 - i)
            yield i + offset

    grid_mels = np.linspace(hertz_to_mels(0), hertz_to_mels(sample_rate), num_filt + 2, True)
    grid_hertz = mel_to_hertz(grid_mels)
    grid_indices = (grid_hertz * fft_len / sample_rate).astype(int)
    grid_indices = list(correct_grid(grid_indices))
    banks = np.zeros([num_filt, fft_len])

    for i, (left, middle, right) in enumerate(chop_array(grid_indices, 3, 1)):
        banks[i, left:middle] = np.linspace(0., 1., middle - left, False)
        banks[i, middle:right] = np.linspace(1., 0., right - middle, False)
    return banks

def wav2mfcc(file_path):
#     print(file_path)
    wave = wavio.read(file_path)
    wave = np.squeeze(wave.data)

    if len(wave) < 7936:
        wave = np.pad(wave, pad_width=(0, 7936-len(wave)), mode='constant')
    elif len(wave) > 7936:
        wave = wave[-7936:]

    return frames2mfcc(wave)

f=open("frames.txt",'w')

In [18]:
def frames2mfcc(wave):
    a=[]
    frames = chop_array(wave, 256, 256)
    h=str(frames)
    h=h.strip()
    h=(h.replace('[array(','').replace('dtype=int16), array(','\n').replace(', dtype=int16)]',''))
    
    f.write(h)
    count =0
    filter_values = []
    for x in range(0, (len(frames))):
        frames1 = np.short(frames[x])
        fft = np.fft.fft(frames1)
        fft = np.array(np.int32(fft.real))
        fft = fft>>7
        fft = np.short(fft)
        sqr = (fft.astype("int32")**2)
        sqr = sqr>>8#(sqr>>8)
        sqr[sqr >= 32768] =  32767
        filters = filterbanks(8000, 16, 256)
        intfltrs = np.array(filters*64)
        intfltrs = intfltrs.astype("int8")     
        mels = np.array(np.dot(sqr.astype("int32"), intfltrs.T.astype("int32")))
#         print("Mels :",mels.shape)
        mfccs = np.array((safe_log(mels)-14), dtype=np.int16)
#         print("Mfccs :",mfccs.shape)
        a.append(mfccs) 
    return a

In [19]:
DATA_PATH = './Training_data/'
folder = 'npy-files'


def get_labels(path=DATA_PATH):
    labels = os.listdir(path)
    labels_indices = np.arange(0, len(labels))
    return labels, labels_indices, to_categorical(labels_indices)

def save_data_to_array(path=DATA_PATH):
    labels, _, _ = get_labels(path)
    for label in labels:
        mfcc_vectors = []
        wavfiles = [path + label + '/' + wavfile for wavfile in os.listdir(path + '/' + label)]
        for wavfile in tqdm(wavfiles):
            mfcc = wav2mfcc(wavfile)
            print(len(mfcc))
            mfcc_vectors.append(mfcc)
        np.save(folder + '/' + label + '.npy', mfcc_vectors)

In [20]:
save_data_to_array()

  0%|                                                                                         | 0/1442 [00:00<?, ?it/s]C:\Users\Ravindra Singh\anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in log2
  
  0%|▏                                                                                | 4/1442 [00:00<01:26, 16.63it/s]

31
31
31
31


  1%|▍                                                                                | 8/1442 [00:00<01:30, 15.83it/s]

31
31
31
31


  1%|▋                                                                               | 12/1442 [00:00<01:26, 16.50it/s]

...

31
31
31
31


 99%|██████████████████████████████████████████████████████████████████████████████▉ | 149/151 [00:09<00:00, 16.32it/s]

31
31
31
31


100%|████████████████████████████████████████████████████████████████████████████████| 151/151 [00:09<00:00, 16.37it/s]

31
